# Module 23: ADAS Capstone - Calibration & Validation


Capstone focus is **ADAS implementation quality**, not coding style.

Your job:
- select a policy profile,
- run test scenarios,
- tune variables,
- justify decisions with safety logic.


In [ ]:

import sys, os, time, cv2
from IPython.display import display, Image, clear_output
sys.path.append(os.path.abspath('../'))
from student_api import AIStudentAPI, VESCStudentAPI

PROFILES = {
    'conservative': {'warning_ttc': 3.8, 'critical_ttc': 1.4, 'warn_brake_a': 2.0, 'crit_brake_a': 5.0},
    'balanced':     {'warning_ttc': 3.0, 'critical_ttc': 1.0, 'warn_brake_a': 2.5, 'crit_brake_a': 6.0},
    'aggressive':   {'warning_ttc': 2.3, 'critical_ttc': 0.8, 'warn_brake_a': 3.0, 'crit_brake_a': 7.0},
}

ACTIVE_PROFILE = 'balanced'  # change only this string and re-run this cell

profile = PROFILES[ACTIVE_PROFILE]
TARGET_LABEL = 'person'
MIN_CONFIDENCE = 0.70
KNOWN_HEIGHT_M = 1.70
FOCAL_PX = 700.0
WHEEL_RADIUS_M = 0.04
GEAR_RATIO = 1.0
WARNING_BRAKE_RAMP_S = 4.0
CRITICAL_BRAKE_RAMP_S = 3.0
ALERT_COOLDOWN_S = 3.0
RUN_DELAY_S = 0.10

ai_api = AIStudentAPI()
if not ai_api.start_camera():
    raise RuntimeError('Camera failed to start. Re-run this cell.')
if not ai_api.start_buzzer():
    raise RuntimeError('Failed to initialize buzzer on GPIO17')

vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    deadline = time.time() + 10.0
    controllers = []
    while time.time() < deadline and not controllers:
        controllers = vesc_api.get_connected_controllers()
        if not controllers:
            time.sleep(0.5)
    if controllers:
        vesc = vesc_api.get_controller(controllers[0])
if not vesc:
    raise RuntimeError('No VESC controller found. This capstone requires brake intervention.')

metrics = {'warning_events': 0, 'critical_events': 0, 'brake_commands': 0}
print('✅ Capstone ready with profile:', ACTIVE_PROFILE)
print('Profile settings:', profile)


In [ ]:

    def rpm_to_speed_mps(rpm):
        wheel_circ = 2.0 * 3.14159 * WHEEL_RADIUS_M
        return abs(rpm) * wheel_circ / 60.0 / max(GEAR_RATIO, 0.01)

    def estimate_distance_m(box_h):
        return (KNOWN_HEIGHT_M * FOCAL_PX) / max(float(box_h), 1.0)

    last_action_time = 0.0

    try:
        print('Running capstone scenario... press Interrupt (■) to stop.')
        while True:
            frame, detections = ai_api.get_frame_and_detections()
            rpm = vesc.get_rpm() or 0.0
            speed_mps = rpm_to_speed_mps(rpm)

            nearest_dist = float('inf')
            for det in detections:
                x, y, w, h = det['box']
                label = det['label']
                conf = float(det['confidence'])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (4, 23, 115), 2)
                cv2.putText(frame, f"{label}:{conf:.2f}", (x, max(15, y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (4, 23, 115), 2)
                if label == TARGET_LABEL and conf >= MIN_CONFIDENCE:
                    nearest_dist = min(nearest_dist, estimate_distance_m(h))

            ttc = nearest_dist / speed_mps if speed_mps > 0.05 and nearest_dist < float('inf') else float('inf')
            now = time.time()
            status = 'SAFE'

            if ttc <= profile['critical_ttc']:
                status = 'CRITICAL'
                if now - last_action_time >= ALERT_COOLDOWN_S:
                    ai_api.buzzer_beep(0.08, 0.08, 3)
                    vesc.set_brake_current(profile['crit_brake_a'], CRITICAL_BRAKE_RAMP_S)
                    metrics['critical_events'] += 1
                    metrics['brake_commands'] += 1
                    last_action_time = now
            elif ttc <= profile['warning_ttc']:
                status = 'WARNING'
                if now - last_action_time >= ALERT_COOLDOWN_S:
                    ai_api.buzzer_beep(0.12, 0.08, 1)
                    vesc.set_brake_current(profile['warn_brake_a'], WARNING_BRAKE_RAMP_S)
                    metrics['warning_events'] += 1
                    metrics['brake_commands'] += 1
                    last_action_time = now

            line1 = f"Profile={ACTIVE_PROFILE} | Status={status} | RPM={rpm:.0f} | Speed={speed_mps:.2f} m/s"
            line2 = f"Dist={nearest_dist if nearest_dist < float('inf') else -1:.2f} m | TTC={ttc if ttc < float('inf') else -1:.2f} s"
            line3 = f"Warnings={metrics['warning_events']} | Critical={metrics['critical_events']} | Brakes={metrics['brake_commands']}"

            cv2.putText(frame, line1, (10, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.48, (255, 255, 255), 2)
            cv2.putText(frame, line2, (10, 44), cv2.FONT_HERSHEY_SIMPLEX, 0.48, (255, 255, 255), 2)
            cv2.putText(frame, line3, (10, 66), cv2.FONT_HERSHEY_SIMPLEX, 0.48, (255, 255, 255), 2)

            _, img_encoded = cv2.imencode('.jpeg', frame)
            clear_output(wait=True)
            display(Image(data=img_encoded.tobytes()))
            print(line1)
            print(line2)
            print(line3)

            time.sleep(RUN_DELAY_S)

    except KeyboardInterrupt:
        print('
Stopped.')
        print('Final metrics:', metrics)



## Capstone Deliverable (ADAS, not code)

Submit:
1. profile selected and final tuned values,
2. 3 scenario results (safe, warning, critical),
3. justification of thresholds and brake policy,
4. one failure case and mitigation plan.


In [ ]:

if 'ai_api' in locals():
    ai_api.shutdown_hardware(stop_camera=True, stop_buzzer=True)
    print('Camera + buzzer released.')
if 'vesc_api' in locals() and vesc_api.is_running():
    vesc_api.stop()
    print('VESC API closed.')
